In [52]:
import numpy as np
import matplotlib.pyplot as plt

def grad(x,function, h):
    n = len(x)
    fprime = np.zeros(n)
    for i in range(n):
        x_aux = np.copy(x)
        x_aux[i] = x[i]+h
        delante = function(x)
        x_aux[i] = x[i]-2*h
        atras = function(x_aux)
        fprime[i] = (delante-atras)/(2*h)
    return fprime

def hessian(x, function, h):
    n = len(x)
    hessian = np.zeros([n, n])
    x_aux = np.copy(x)
    
    for i in range(n):
        for j in range(n):
            x[i] = x[i]+h 
            x[j] = x[j]+h
            d = function(x)
            x[j] = x[j]-2*h
            a = function(x)
            #regresar al valor de x original
            x = np.copy(x_aux)

            x[i] = x[i]-h 
            x[j] = x[j]+h
            dd = function(x)
            x[j] = x[j]-2*h
            aa = function(x)
            #regresar al original
            x = np.copy(x_aux)
            
            #llenar matriz
            hessian[i][j] = (d-a-dd+aa)/(4*h*h)

    return hessian

def rosenbrock(x):
    n = len(x)
    suma = 0
    for i in range(n-1):
        suma += 100*(x[i+1]-x[i]**2)**2+(1-x[i])**2
    return suma

def wood(x):
    return sum((
        100*(x[0]*x[0] - x[1])**2,
        (x[0]-1)**2,
        (x[2]-1)**2,
        90*(x[2]*x[2] - x[3])**2,
        10.1*((x[1]-1)**2 + (x[3]-1)**2),
        19.8*(x[1]-1)*(x[3]-1),
        ))

def branin(x):
    a = 1.0
    b = 5.1 / (4*np.pi**2)
    c = 5.0 / (np.pi)
    r = 6.0
    s = 10.0
    t = 1.0 / (8*np.pi)
    
    return a*(x[1]-b*x[0]**2+c*x[0]-r)**2+s*(1-t)*np.cos(x[0])+s

In [3]:
def dogleg_method(funcion, x_k, h, alpha, delta, eta, tol, max_iter):
    for i in range(max_iter):
        f_k = funcion(x_k)
        g = rosen_der(x_k)#grad(x_k, funcion, h)
        B = rosen_hess(x_k)#hessian(x_k, funcion, h)    

        '''1) Minimizar a lo largo del gradiente'''
        semi_pos_cond = np.dot(np.dot(g.T, B), g)
        pU = -(np.dot(g.T, g))*g/(semi_pos_cond)
        pB = -np.dot(np.linalg.inv(B), g)
        '''2) Minimizar si el hessiano es semidefinido positivo'''
        norm_grad = np.linalg.norm(g)
        #print("norma grad: ", norm_grad)
        if (semi_pos_cond > 0):
            #print("si")
            a = np.linalg.norm(pB-pU)**2
            b = 2*np.dot((pB.T),(pB-pU))
            c = np.linalg.norm(pU)**2-delta**2
            coeffs = [a, b, c]
            roots = np.roots(coeffs)
            
            tau1 = roots[0]+1
            tau2 = roots[1]+1
            
            #print(tau1, tau2)
            if (0 <= tau1 <= 2):
                if (0 <= tau2 <= 2):
                    tau = min(tau1, tau2)
            elif (0 <= tau2 <= 2):
                tau = tau2
            #else:
            #    tau = 1.0
            
            
            if (tau <= 1):
                p_k = tau*pU
            else:
                p_k = pU+(tau-1)*(pB-pU)
            
            
            rho_num = rosen(x_k)-rosen(x_k+p_k)
            rho_den = -np.dot(g.T, p_k)-(0.5)*np.dot(np.dot(p_k.T,B), p_k)
            
            rho = rho_num / rho_den
            #print(rho)
            '''3) Calcular el tamano de paso'''
            if (rho > eta):
                #print("si eta")
                x_k += p_k
            delta = np.sqrt(np.linalg.norm(pU+(tau-1)*(pB-pU))**2)
            if (norm_grad < tol):
                break
    return x_k
        #minimizar el modelo cuadraticon sin restricciones a lo largo del gradiente

## Rosenbrock function n = 100

In [6]:
from scipy.optimize import rosen, rosen_der, rosen_hess
#Condiciones 
n = 100
x_k = np.ones(n) + np.random.uniform(-2, 2, n)
max_iter = 5000
alpha = 0.02
delta = 2
eta = 0.1
tol = 0.01
h = 0.000001
res = np.ones(n)
mean = np.zeros(30)

for i in range(30):
    print(i)
    x_k = np.ones(n) + np.random.uniform(-2, 2, n)
    x = dogleg_method(rosen, x_k, h, alpha, delta, eta, tol, max_iter)
    mean[i] = (x[0]-res).mean()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29


In [8]:
mean.mean()

-0.30956923475680004

## Wood function

In [36]:
def dogleg_method(funcion, h, x_k, alpha, delta, eta, tol, max_iter):
    n = len(x_k)
    for i in range(max_iter):
        f_k = funcion(x_k)
        g = grad(x_k, funcion, h)
        B = hessian(x_k, funcion, h)    

        '''1) Minimizar a lo largo del gradiente'''
        semi_pos_cond = np.dot(np.dot(g.T, B), g)
        #print("cond: ", semi_pos_cond)
        pU = -(np.dot(g.T, g))*g/(semi_pos_cond)
        pB = -np.dot(np.linalg.inv(B), g)
        '''2) Minimizar si el hessiano es semidefinido positivo'''
        norm_grad = np.linalg.norm(g)
        #print("norma grad: ", norm_grad)
        if (semi_pos_cond > 0):
            #print("si")
            a = np.linalg.norm(pB-pU)**2
            b = 2*np.dot((pB.T),(pB-pU))
            c = np.linalg.norm(pU)**2-delta**2
            coeffs = [a, b, c]
            roots = np.roots(coeffs)
            
            tau1 = roots[0]+1
            tau2 = roots[1]+1
            
            #print(tau1, tau2)
            if (0 <= tau1 <= 2):
                if (0 <= tau2 <= 2):
                    tau = min(tau1, tau2)
            elif (0 <= tau2 <= 2):
                tau = tau2
            else:
                tau = 1.0
            
            
            if (tau <= 1):
                p_k = tau*pU
            else:
                p_k = pU+(tau-1)*(pB-pU)
            
            
            rho_num = funcion(x_k)-funcion(x_k+p_k)
            rho_den = -np.dot(g.T, p_k)-(0.5)*np.dot(np.dot(p_k.T,B), p_k)
            
            rho = rho_num / rho_den
            #print(rho)
            '''3) Calcular el tamano de paso'''
            if (rho > eta):
                #print("si eta")
                x_k += p_k
            delta = np.sqrt(np.linalg.norm(pU+(tau-1)*(pB-pU))**2)
            if (norm_grad < tol):
                break
    return x_k
        #minimizar el modelo cuadraticon sin restricciones a lo largo del gradiente

In [37]:
n = 4
x_k = np.ones(n) + np.random.uniform(-2, 2, n)
max_iter = 100
alpha = 0.02
delta = 2
eta = 0.1
tol = 0.01
h = 0.000001
res = np.ones(n)
mean = np.zeros(30)

for i in range(30):
    x_k = np.ones(n) + np.random.uniform(-2, 2, n)
    x = dogleg_method(wood, h, x_k, alpha, delta, eta, tol, max_iter)
    mean[i] = (x[0]-res).mean()

In [39]:
mean.mean()

-0.44862995359514657

## Función Branin

In [60]:
n = 2
x_k = np.zeros(n)
x_k[0] = np.pi + np.random.uniform(-2, 2)
x_k[1] = 2.275 + np.random.uniform(-2, 2)

max_iter = 1000
alpha = 0.02
delta = 2
eta = 0.1
tol = 0.01
h = 0.000001
res = np.zeros(n)
res[0] = np.pi
res[1] = 2.275

mean = np.zeros(30)

for i in range(30):
    print(i)
    x_k = np.zeros(n)
    x_k[0] = np.pi + np.random.uniform(-2, 2)
    x_k[1] = 2.275 + np.random.uniform(-2, 2)
    
    x = dogleg_method(branin, h, x_k, alpha, delta, eta, tol, max_iter)
    mean[i] = (x[0]-res).mean()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29


In [63]:
mean.mean()

0.7012817599789134